In [3]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc

In [4]:
#Load datasets 
sales_data = pd.read_csv(r'C:\projects\retail-sales-analytics\data\raw\sales.csv')
customer_data=pd.read_csv(r'C:\projects\retail-sales-analytics\data\raw\customers.csv')
products_data = pd.read_csv(r'C:\projects\retail-sales-analytics\data\raw\products.csv')
inventory_data=pd.read_csv((r'C:\projects\retail-sales-analytics\data\raw\inventory.csv'))

In [5]:
#confirm datasets are loaded
print(sales_data.head())
print(customer_data.head())
print(products_data.head())     
print(inventory_data.head())

   transaction_id  store_id  product_id  customer_id  quantity  price  \
0               1       102        1004       5030.0       100  31.63   
1               2       103        1002       5032.0         5  13.34   
2               3       103        1003          NaN         1  49.18   
3               4       103        1001       5039.0        10  45.96   
4               5       103        1001       5044.0       100  17.94   

         date  
0  11-03-2025  
1  2025-12-05  
2  2025-04-28  
3  2025-12-23  
4  14-09-2025  
   customer_id   name gender customer_type
0         5001  David    NaN       regular
1         5002  Shane    NaN       Regular
2         5003  Tammy      F           VIP
3         5004  Aimee    NaN           vip
4         5005  Norma    NaN           VIP
   product_id    product_name     category  price
0        1001        Notebook   Stationery   10.5
1        1002             Pen   Stationery   20.0
2        1003  USB Drive 16GB  Electronics   15.0
3      

Sales Data

In [6]:
#inspect sales data
sales_data.info()
##sales_data has 500 rows and 7 columns
##The data shows that there is missing values in customer_id
sales_data.isnull().sum()
##The data shows that we have 44 missing values in customer_id column
sales_data.dtypes
##customer_id is float type, should be int type and date column is object type currently,should be datetime type
sales_data.duplicated().sum()
##no duplicate rows found in sales_data
sales_data.describe()
##The data shows that quantity column has a negative minimum value of -1 which will need to be checked further to see if its a return or an error

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  500 non-null    int64  
 1   store_id        500 non-null    int64  
 2   product_id      500 non-null    int64  
 3   customer_id     456 non-null    float64
 4   quantity        500 non-null    int64  
 5   price           500 non-null    float64
 6   date            500 non-null    object 
dtypes: float64(2), int64(4), object(1)
memory usage: 27.5+ KB


,transaction_id,store_id,product_id,customer_id,quantity,price
count,500.000000,500.000000,500.000000,456.000000,500.000000,500.00000
mean,250.500000,102.068000,1003.002000,5025.774123,17.822000,28.33366
std,144.481833,0.807887,1.420575,14.560502,34.716463,12.99169
min,1.000000,101.000000,1001.000000,5001.000000,-1.000000,5.16000
25%,125.750000,101.000000,1002.000000,5013.000000,1.000000,17.29000
50%,250.500000,102.000000,1003.000000,5026.000000,3.000000,28.41500
75%,375.250000,103.000000,1004.000000,5039.000000,10.000000,40.03250
max,500.000000,103.000000,1005.000000,5050.000000,100.000000,49.99000


In [7]:
#Sales data cleaning
sales_data.dtypes
##convert date column to datetime type
sales_data['date']=pd.to_datetime(sales_data['date'],format='mixed',errors='coerce')
sales_data['date'].isna().sum()
##standardized date column
sales_data['date'] = pd.to_datetime(sales_data['date'],format='%Y-%m-%d')
sales_data['date'].head(20)
##date column is now standardized
sales_data.dtypes
##convert customer_id to int type
sales_data['customer_id']=sales_data['customer_id'].fillna(0).astype(int)
##missing values in customer_id filled with 0 and converted to int type.after conversion 0 values represent missing customer_id
sales_data.isnull().sum()
## dtype final check
sales_data.dtypes
##customer_id is now int type and date is datetime type
#investigate negative quantity values
sales_data[sales_data['quantity']<0]
##the negative quantity values represent returns and will be kept as is for analysis with a total of 82 rows affected



,transaction_id,store_id,product_id,customer_id,quantity,price,date
5,6,103,1001,5014,-1,19.91,2025-09-21
15,16,102,1004,5044,-1,12.26,2025-07-10
21,22,103,1001,5008,-1,10.70,2025-06-30
22,23,103,1003,5050,-1,17.21,2025-07-19
23,24,101,1002,0,-1,14.69,2025-01-01
...,...,...,...,...,...,...,...
490,491,102,1005,5046,-1,31.90,2025-05-10
491,492,102,1005,5050,-1,45.56,2025-11-25
492,493,103,1003,5021,-1,26.89,2025-03-31
493,494,102,1004,5049,-1,34.13,2025-08-06


In [8]:
#consolidate cleaned sales data
sales_staged=sales_data.copy()
sales_staged.head()

,transaction_id,store_id,product_id,customer_id,quantity,price,date
0,1,102,1004,5030,100,31.63,2025-11-03
1,2,103,1002,5032,5,13.34,2025-12-05
2,3,103,1003,0,1,49.18,2025-04-28
3,4,103,1001,5039,10,45.96,2025-12-23
4,5,103,1001,5044,100,17.94,2025-09-14


Customers Data

In [24]:
#inspect customer data
customer_data.info()
##customer_data has 50 rows and 4 columns
## The data shows that we have missing values in gender column
customer_data.isnull().sum()
## The data shows that we have 16 missing values in gender column
customer_data.dtypes
##there are no incorrect data types
customer_data.duplicated().sum()
##no duplicate rows found in customer_data
customer_data.describe()
## no anomalies found in customer_data
customer_data.head(20)
##customer gender shows that they are Nan values representing unknown gender values to check further to accertan if they are part of the calculated missing values
## customer type column shows inconsistent capitalization of values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    50 non-null     int64 
 1   name           50 non-null     object
 2   gender         50 non-null     object
 3   customer_type  50 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


,customer_id,name,gender,customer_type
0,5001,David,Unknown,Regular
1,5002,Shane,Unknown,Regular
2,5003,Tammy,F,Vip
3,5004,Aimee,Unknown,Vip
4,5005,Norma,Unknown,Vip
5,5006,Donna,Unknown,Regular
6,5007,Amanda,M,Vip
7,5008,Maria,F,Regular
8,5009,Cynthia,M,Regular
9,5010,Elijah,M,Regular


In [22]:
#Customer data cleaning
##handle missing gender values
customer_data['gender']=customer_data['gender'].fillna('Unknown')
customer_data.isnull().sum()
##missing values in gender column have been handled as 'Unknown'
##standardize customer type values
customer_data['customer_type']= customer_data['customer_type'].str.capitalize()
customer_data['customer_type'].unique()
customer_data.head(20)
##customer type values have been standardized to have consistent capitalization

,customer_id,name,gender,customer_type
0,5001,David,Unknown,Regular
1,5002,Shane,Unknown,Regular
2,5003,Tammy,F,Vip
3,5004,Aimee,Unknown,Vip
4,5005,Norma,Unknown,Vip
5,5006,Donna,Unknown,Regular
6,5007,Amanda,M,Vip
7,5008,Maria,F,Regular
8,5009,Cynthia,M,Regular
9,5010,Elijah,M,Regular


In [23]:
##cutomer data staged
customer_staged=customer_data.copy()
customer_staged.head()

,customer_id,name,gender,customer_type
0,5001,David,Unknown,Regular
1,5002,Shane,Unknown,Regular
2,5003,Tammy,F,Vip
3,5004,Aimee,Unknown,Vip
4,5005,Norma,Unknown,Vip


Inventory Data

In [ ]:
#inspect inventory data
inventory_data.info()
##inventory_data has 18 rows and 5 columns
## stock_quantity column shows missing values
inventory_data.isnull().sum()
## stock_quantity column has 3 missing values
inventory_data.dtypes
## suplier_id column should be int type instead of object type
inventory_data.duplicated().sum()
#3 duplicate rows found in inventory_data
inventory_data.describe()
## no anomalies found in inventory_data
inventory_data.head(20)
##supplier_id is a categorical identifier and should remain a string, but needs standardization(e.g S01,s01,s02,S02)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   store_id        18 non-null     int64  
 1   product_id      18 non-null     int64  
 2   stock_quantity  15 non-null     float64
 3   reorder_level   18 non-null     int64  
 4   supplier_id     18 non-null     object 
dtypes: float64(1), int64(3), object(1)
memory usage: 852.0+ bytes


,store_id,product_id,stock_quantity,reorder_level,supplier_id
0,101,1001,10.0,20,S02
1,101,1002,20.0,15,S03
2,101,1003,10.0,20,s03
3,101,1004,50.0,5,S02
4,101,1005,0.0,5,s02
5,102,1001,NaN,5,s03
6,102,1002,0.0,15,S01
7,102,1003,20.0,10,s02
8,102,1004,0.0,15,S02
9,102,1005,10.0,20,s03


In [ ]:
#Inventory data cleaning
##handle missing stock_quantity values
inventory_data.isnull().sum()
## stock_quantity column has 3 missing values which will be flagged as unknown and will be left as is for analysis
inventory_data.dtypes
#supplier_id is a categorical identifier and should remain a string, but needs standardization
##remove duplicate rows
inventory_data=inventory_data.drop_duplicates()
inventory_data.duplicated().sum()
##duplicate rows have been removed
##standardize supplier_id values
inventory_data['supplier_id']=inventory_data['supplier_id'].str.upper()
inventory_data['supplier_id'].unique()
##supplier_id values have been standardized to uppercase
#final check
inventory_data.head(20)


,store_id,product_id,stock_quantity,reorder_level,supplier_id
0,101,1001,10.0,20,S02
1,101,1002,20.0,15,S03
2,101,1003,10.0,20,S03
3,101,1004,50.0,5,S02
4,101,1005,0.0,5,S02
5,102,1001,NaN,5,S03
6,102,1002,0.0,15,S01
7,102,1003,20.0,10,S02
8,102,1004,0.0,15,S02
9,102,1005,10.0,20,S03


In [43]:
#inventory data staged
inventory_staged=inventory_data.copy()
inventory_staged.head()

,store_id,product_id,stock_quantity,reorder_level,supplier_id
0,101,1001,10.0,20,S02
1,101,1002,20.0,15,S03
2,101,1003,10.0,20,S03
3,101,1004,50.0,5,S02
4,101,1005,0.0,5,S02


Products data

In [ ]:
#inspect products data
products_data.info()
##products_data has 5 rows and 4 columns
## The data shows missing values in category column
products_data.isnull().sum()
## category column has 1 missing value
products_data.dtypes
## no incorrect data types found
products_data.duplicated().sum()
## no duplicate rows found in products_data
products_data.describe()
## no anomalies found in products_data
products_data.head(20)
## category column has a missing value to check further

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    5 non-null      int64  
 1   product_name  5 non-null      object 
 2   category      4 non-null      object 
 3   price         5 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 292.0+ bytes


,product_id,product_name,category,price
0,1001,Notebook,Stationery,10.5
1,1002,Pen,Stationery,20.0
2,1003,USB Drive 16GB,Electronics,15.0
3,1004,Headphones,Electronics,45.0
4,1005,Charger,NaN,25.0


In [ ]:
#products data cleaning
#handle missing category value
products_data.isnull().sum()
products_data['category']=products_data['category'].fillna('Electronics')
#missing value in category column has been handled as 'Electronics'since we have similar products in that category


In [52]:
#staged products data
products_staged=products_data.copy()        
products_staged.head()

,product_id,product_name,category,price
0,1001,Notebook,Stationery,10.5
1,1002,Pen,Stationery,20.0
2,1003,USB Drive 16GB,Electronics,15.0
3,1004,Headphones,Electronics,45.0
4,1005,Charger,Electronics,25.0


In [78]:
# Save cleaned staged data to CSV
# Ensure the folder exists
import os
os.makedirs(r'C:\projects\retail-sales-analytics\data\processed', exist_ok=True)

# Save cleaned staged data into the processed folder
sales_staged.to_csv(r'C:\projects\retail-sales-analytics\data\processed\sales_staged.csv', index=False)
customer_staged.to_csv(r'C:\projects\retail-sales-analytics\data\processed\customer_staged.csv', index=False)
products_staged.to_csv(r'C:\projects\retail-sales-analytics\data\processed\products_staged.csv', index=False)
inventory_staged.to_csv(r'C:\projects\retail-sales-analytics\data\processed\inventory_staged.csv', index=False)
